# API para la exracción de data desde Google Ads

### Ejemplo data de campañas VTR

#### Instalación de librerias

In [1]:
#pip install google
#!pip install --upgrade google-api-python-client
#!pip install google-ads
#!pip install pandas-gbq

#### Aumentamos el limite de filas para visualizar en dataframe

In [57]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

#### Librerias

In [1]:
import argparse
from ensurepip import version
import sys
import os

from google.ads.googleads.client import GoogleAdsClient
from google.ads.googleads.errors import GoogleAdsException
from google.oauth2 import service_account
import pandas as pd
import pandas_gbq

#### Agregamos las credenciales de google y el ID de cliente

In [2]:
# Credenciales google
os.environ["GOOGLE_ADS_CONFIGURATION_FILE_PATH"] = "credentials.yaml"
customerId = "8530437202"
client = GoogleAdsClient.load_from_storage(version="v11")

#### codigo de extracción de data y documentación

1. [Query builder google ads api](https://developers.google.com/google-ads/api/fields/v11/query_validator)

2. [Lista de metricas google ads api](https://developers.google.com/google-ads/api/fields/v11/metrics#metrics.active_view_measurable_cost_micros)
3. [Lista de dimensiones google ads api](https://developers.google.com/google-ads/api/fields/v11/overview)

In [87]:
# Obtener data a partir de una consulta SQL
def getData(customerId):
    gaService = client.get_service('GoogleAdsService')

    query = """
        SELECT
          campaign.id,
          campaign.name,
          campaign.start_date,
          campaign.advertising_channel_type,
          segments.device,
          metrics.impressions,
          metrics.clicks,
          metrics.cost_micros
        FROM campaign
        WHERE campaign.start_date > '2022-01-02' AND campaign.start_date < '2022-07-31' 
        ORDER BY campaign.start_date desc"""

    stream = gaService.search_stream(customer_id=customerId, query=query)
    
    return stream

# Limpieza de data 
def cleanData():
    dataDicc = []
    dataCampaign = getData(customerId)

    for batch in dataCampaign:
        for row in batch.results:
            dataDicc.append({
                'id': row.campaign.id,
                'campaign name': row.campaign.name,
                'advertising': row.campaign.advertising_channel_type,
                'device': row.segments.device,
                'date': row.campaign.start_date,
                #'end_date': row.campaign.end_date,
                'clicks': row.metrics.clicks,
                'impressions': row.metrics.impressions,
                'cost': row.metrics.cost_micros
            })
    
    df = pd.DataFrame(dataDicc)
    
    return df

# Enviar data a BigQuery
def uploadDataBQ():
    data = cleanData()
    data.to_gbq(destination_table='test_gads.data_google_ads',
        project_id='has-latam-prod-apicm-paris-cl',
        if_exists='replace')

#if __name__ == "__main__":
 #   print(cleanData())

#### Capturamos en un dataframe la data extraída por la API y visualizamos

In [89]:
data = cleanData()
data = pd.DataFrame(data)
# la metrica costo se obtiene a traves de cost.micros/1000000
data["cost"] = (data["cost"]/1000000).astype("int64")
print(data.shape)
data

(190, 8)


,id,campaign name,advertising,device,date,clicks,impressions,cost
0,17890794440,YT | VTR SamsungS21FE | Awareness | Julio/Agos...,6,4,2022-07-28,36,23155,13639
1,17890794440,YT | VTR SamsungS21FE | Awareness | Julio/Agos...,6,2,2022-07-28,594,83681,55893
2,17890794440,YT | VTR SamsungS21FE | Awareness | Julio/Agos...,6,3,2022-07-28,127,18238,10538
3,17890794440,YT | VTR SamsungS21FE | Awareness | Julio/Agos...,6,6,2022-07-28,37,607215,307166
4,17613991824,Nacional | AO PM | Hogar | Internet | Ecommerc...,10,4,2022-06-29,1171,80481,1136599
5,17613991824,Nacional | AO PM | Hogar | Internet | Ecommerc...,10,2,2022-06-29,13039,934293,4033781
6,17613991824,Nacional | AO PM | Hogar | Internet | Ecommerc...,10,3,2022-06-29,106,15947,40321
7,17613991824,Nacional | AO PM | Hogar | Internet | Ecommerc...,10,6,2022-06-29,1,11022,14955
8,17511138686,YT | Experimento Video | Awareness | Junio 202...,6,5,2022-06-16,0,1,0
9,17511142694,YT | Experimento Video | Awareness | Junio 202...,6,5,2022-06-16,0,5,5
